In [ ]:
import os
import pandas as pd
import math

def magnitude(X, Y, Z):
    return math.sqrt(float(X**2) + float(Y**2) + float(Z**2))


path = 'gestures-dataset'

dataset = None

for subject in os.listdir(path):
    if os.path.isfile(os.path.join(path, subject)):
        continue
    if subject in ('U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08'):
        for gesture in os.listdir(os.path.join(path, subject)):
            if os.path.isfile(os.path.join(path, subject, gesture)):
                continue
            gesture = str(gesture)
            for samplefile in os.listdir(os.path.join(path, subject, gesture)):
                if os.path.isfile(os.path.join(path, subject, gesture, samplefile)):
                    df = pd.read_csv(os.path.join(path, subject, gesture, samplefile), \
                        sep = ' ', \
                        names = ['System.currentTimeMillis()', \
                        'System.nanoTime()', \
                        'sample.timestamp', \
                        'X', \
                        'Y', \
                        'Z' \
                        ])
                    df = df[["sample.timestamp", "X", "Y", "Z"]]
                                        
                    start = df["sample.timestamp"][0]
                    df["sample.timestamp"] -= start
                    df["sample.timestamp"] /= 10000000
                    df["subject"] = subject
                    df["gesture"] = gesture
                    df["sample"] = str(samplefile[:-4])
                    #print(df)
                    if dataset is None:
                        dataset = df.copy()
                    else:
                        dataset = pd.concat([dataset, df])

dataset = dataset.sort_values(by=['gesture','subject','sample','sample.timestamp'])
#print(dataset)
print(dataset.head(10))
print(dataset.tail(10))

                    
            
            

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", None, "display.max_columns", None)

gdf = dataset.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['mean', 'std', 'min', 'max']})
gdf

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset_scaled = None

for i, gesture in enumerate(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']):
    for j, subject in enumerate(['U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08']):
        for k, sample in enumerate(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']):
                    
            df = dataset[dataset['gesture']==gesture]
            df = df[df['subject']==subject]
            df = df[df['sample']==sample]
            df.sort_values(by=['sample.timestamp'])

            sc = scaler
            sc = sc.fit_transform(df[["X", "Y", "Z"]])
            sc = pd.DataFrame(data=sc, columns=["X", "Y", "Z"])
            df["X"] = sc["X"]
            df["Y"] = sc["Y"]
            df["Z"] = sc["Z"]
            #df["magnitude"] = sc.apply(lambda row: magnitude(row['X'], row['Y'], row['Z']), axis=1)
            if dataset_scaled is None:
                dataset_scaled = df.copy()
            else:
                dataset_scaled = pd.concat([dataset_scaled, df])
                
#print(dataset_scaled)
print(dataset_scaled.head(10))
print(dataset_scaled.tail(10))



In [ ]:
import numpy as np

for i, gesture in enumerate(dataset_scaled['gesture'].unique()):
    df_gesture = dataset_scaled[dataset_scaled['gesture']==gesture]
    f, axes = plt.subplots(8, 3, figsize=(50,150), sharex="row", sharey="row")
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
                
        time_max = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['max']})
        #print(time_max)
        time_max = time_max['sample.timestamp'].iloc[0]['max']

        
        time_scale = []
        for tmp in range(0, time_max * 11, 11):
            time_scale.append(tmp)
        #print(time_scale)
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['X']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].count() < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].count() - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,0])
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Y']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].count() < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].count() - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,1])
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Z']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].count() < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].count() - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,2])
        #plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)        
    plt.show()

        

In [ ]:
import numpy as np

dataset_cleaned = None

for i, gesture in enumerate(dataset_scaled['gesture'].unique()):
    df_gesture = dataset_scaled[dataset_scaled['gesture']==gesture]
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
        
        time_mean = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['mean']})
        time_std = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['std']})
        time_max = time_mean['sample.timestamp'].iloc[0]['mean'] + 1.0 * time_std['sample.timestamp'].iloc[0]['std']
        #print(time_max)
        time_min = time_mean['sample.timestamp'].iloc[0]['mean'] - 1.0 * time_std['sample.timestamp'].iloc[0]['std']
        #print(time_min)
        for i, sample in enumerate(df_subject['sample'].unique()):
            df_sample_count = df_subject[df_subject['sample']==sample].count()['sample.timestamp']
            #print(df_sample_count)
            if df_sample_count < time_min or df_sample_count > time_max:
                df_subject = df_subject[df_subject['sample'] != sample]
                
        if dataset_cleaned is None:
            dataset_cleaned = df_subject.copy()
        else:
            dataset_cleaned = pd.concat([dataset_cleaned, df_subject])

print(dataset_cleaned.head(10))
print(dataset_cleaned.tail(10))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", None, "display.max_columns", None)

gdf = dataset_cleaned.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['mean', 'std', 'min', 'max']})
gdf

In [ ]:
import numpy as np

for i, gesture in enumerate(dataset_cleaned['gesture'].unique()):
    df_gesture = dataset_cleaned[dataset_cleaned['gesture']==gesture]
    f, axes = plt.subplots(8, 3, figsize=(50,150), sharex="all", sharey="all")
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
        
        time_max = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['max']})
        #print(time_max)
        time_max = time_max['sample.timestamp'].iloc[0]['max']
        
        time_scale = []
        for tmp in range(0, time_max * 11, 11):
            time_scale.append(tmp)
        #print(time_scale)
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['X']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].size < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].size - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,0])
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Y']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].size < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].size - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,1])
        dictionary = {'time' : pd.Series(time_scale)}
        for i, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Z']
            #print(dictionary[str(sample)].size)
            if dictionary[str(sample)].size < time_max:
                tmp = []
                for i in range((dictionary[str(sample)].size - 1) * 11, time_max - 1 * 11, 11):
                    tmp.append(0.0)
                tmp = pd.Series(tmp,dtype=np.float64)
                dictionary[str(sample)] = dictionary[str(sample)].append(tmp, ignore_index=True)
            #print(dictionary[str(sample)].size)
        #print(dictionary)
        data_preproc = pd.DataFrame(dictionary)
        sns.lineplot(x='time', y='value', hue='variable', data=pd.melt(data_preproc, ['time']),ax=axes[j,2])
        #plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)        
    plt.show()

        

In [ ]:
import numpy as np

dataset_timecut = None

for i, gesture in enumerate(dataset_cleaned['gesture'].unique()):
    df_gesture = dataset_cleaned[dataset_cleaned['gesture']==gesture]
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject] 
        time_max = 19 # 18 * 11 = 198
        for i, sample in enumerate(df_subject['sample'].unique()):
            df_sample = df_subject[df_subject['sample']==sample]
            df_sample_count = df_sample.count()['sample.timestamp']
            #print(df_sample_count)
            if df_sample_count >= time_max:
                df_sample = df_sample[df_sample['sample.timestamp'] <= (11 * (time_max-1))]
                df_sample_count = df_sample.count()['sample.timestamp']
                #print(df_sample_count)
            elif df_sample_count < time_max:
                for tmp in range(df_sample_count * 11, (time_max) * 11, 11):
                    df = pd.DataFrame([[tmp, 0.0, 0.0, 0.0, gesture, subject, sample]], columns=['sample.timestamp', 'X', 'Y', 'Z', 'gesture', 'subject', 'sample'])
                    df_sample = df_sample.append(df, ignore_index=True)            
            #print(df_sample)
            df_sample_count = df_sample.count()['sample.timestamp']
            #print(df_sample_count)
            if df_sample_count != time_max:
                continue
            if dataset_timecut is None:
                dataset_timecut = df_sample.copy()
            else:
                dataset_timecut = pd.concat([dataset_timecut, df_sample])

print(dataset_timecut.head(10))
print(dataset_timecut.tail(10))

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

gdf = dataset_timecut.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['mean', 'std', 'min', 'max']})
gdf

        

In [ ]:
import numpy as np

for i, gesture in enumerate(dataset_timecut['gesture'].unique()):
    df_gesture = dataset_timecut[dataset_timecut['gesture']==gesture]
    f, axes = plt.subplots(8, 3, figsize=(50,150), sharex="all", sharey="all")
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
                        
        time_max = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['max']})
        #print(time_max)
        time_max = time_max['sample.timestamp'].iloc[0]['max']

        time_scale = []
        for tmp in range(0, time_max * 11, 11):
            time_scale.append(tmp)

        dictionary = {'time' : pd.Series(time_scale)}
        for k, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['X']
        data_preproc = pd.DataFrame(dictionary)
        data=pd.melt(data_preproc, ['time'])
        sns.lineplot(x='time', y='value', hue='variable', data=data, ax=axes[j,0])

        dictionary = {'time' : pd.Series(time_scale)}
        for k, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Y']
        data_preproc = pd.DataFrame(dictionary)
        data=pd.melt(data_preproc, ['time'])
        sns.lineplot(x='time', y='value', hue='variable', data=data, ax=axes[j,1])

        dictionary = {'time' : pd.Series(time_scale)}
        for k, sample in enumerate(df_subject['sample'].unique()):
            dictionary[str(sample)] = df_subject[df_subject['sample']==sample]['Z']
        data_preproc = pd.DataFrame(dictionary)
        data=pd.melt(data_preproc, ['time'])
        sns.lineplot(x='time', y='value', hue='variable', data=data, ax=axes[j,2])

    plt.show()

        

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))

sns.countplot(x = 'subject',
              data = dataset_timecut,
              palette=[sns.color_palette()[0]]
              );
plt.title("Records per subject");
plt.show()

plt.figure(figsize=(10,10))
sns.countplot(x = 'gesture',
              data = dataset_timecut,
              palette=[sns.color_palette()[0]]
              );
plt.title("Records per gesture");
plt.show()

In [ ]:
import numpy as np
import math

features_sample = None

for i, gesture in enumerate(dataset_timecut['gesture'].unique()):
    df_gesture = dataset_timecut[dataset_timecut['gesture']==gesture]
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
        for k, sample in enumerate(df_subject['sample'].unique()):
            df_sample = df_subject[df_subject['sample']==sample]
            df_sample.sort_values(by=['sample.timestamp'])

            df_feature = pd.DataFrame(columns = ["gesture","subject", "sample"])
            df_feature = df_feature.append({'gesture' :gesture, 'subject' : subject, 'sample' : sample, \
                                        'meanx': df_sample["X"].mean(), 'meany': df_sample["Y"].mean(), 'meanz': df_sample["Z"].mean(), \
                                        'stdx': df_sample["X"].std(), 'stdy': df_sample["Y"].std(), 'stdz': df_sample["Z"].std(), \
                                        'madx': df_sample["X"].mad(), 'mady': df_sample["Y"].mad(), 'madz': df_sample["Z"].mad(), \
                                        'semx': df_sample["X"].sem(), 'semy': df_sample["Y"].sem(), 'semz': df_sample["Z"].sem(), \
                                        'kurtx': df_sample["X"].kurt(), 'kurty': df_sample["Y"].kurt(), 'kurtz': df_sample["Z"].kurt(), \
                                        'skewx': df_sample["X"].skew(), 'skewy': df_sample["Y"].skew(), 'skewz': df_sample["Z"].skew(), \
                                        'corr9x': df_sample["X"].autocorr(9), 'corr9y': df_sample["Y"].autocorr(9), 'corr9z': df_sample["Z"].autocorr(9), \
                                        'corr6x': df_sample["X"].autocorr(6), 'corr9y': df_sample["Y"].autocorr(6), 'corr9z': df_sample["Z"].autocorr(6), \
                                        'corr3x': df_sample["X"].autocorr(3), 'corr9y': df_sample["Y"].autocorr(3), 'corr9z': df_sample["Z"].autocorr(3), \
                                        'corr2x': df_sample["X"].autocorr(2), 'corr9y': df_sample["Y"].autocorr(2), 'corr9z': df_sample["Z"].autocorr(2), \
                                       }, \
                                       ignore_index=True)
            if features_sample is None:
                features_sample = df_feature.copy()
            else:
                features_sample = pd.concat([features_sample, df_feature], ignore_index=True)

print(features_sample.head(10))
print(features_sample.tail(10))


In [3]:
import sys
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit
from scipy.stats import pearsonr
from matplotlib import pyplot
import sklearn 
#print(sklearn.__version__)
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedGroupKFold

# create the dataset
def get_dataset():
    X = features_sample.copy().drop(labels = ["gesture","subject", "sample"], axis = 1).to_numpy()
    y = features_sample.copy()["gesture"].to_list()
    g = features_sample.copy()["subject"].to_list()
    return X, y, g
 
# retrieve the model to be evaluate
def get_model():
	model = LogisticRegression()
	return model

# get a list of models to evaluate
def get_models():
	models = list()
	models.append(LogisticRegression(penalty="l2",max_iter=100000,solver="sag"))
	models.append(RidgeClassifier(max_iter=100000, solver="sag"))
	models.append(SGDClassifier(max_iter=100000))
	models.append(PassiveAggressiveClassifier(max_iter=100000))
	models.append(KNeighborsClassifier(n_neighbors=5))
	models.append(DecisionTreeClassifier())
	models.append(ExtraTreeClassifier())
	models.append(LinearSVC(max_iter=100000))
	models.append(SVC())
	models.append(GaussianNB())
	models.append(AdaBoostClassifier())
	models.append(BaggingClassifier())
	models.append(RandomForestClassifier())
	models.append(ExtraTreesClassifier())
	models.append(GaussianProcessClassifier(max_iter_predict=10000))
	models.append(GradientBoostingClassifier())
	models.append(LinearDiscriminantAnalysis())
	models.append(QuadraticDiscriminantAnalysis())
	return models
 
# evaluate the model using a given test condition
def evaluate_model(cv, model):
    # get the dataset
    X, y, g = get_dataset()
    cv = cv.split(X, y, g)
    # evaluate the model
    scores = cross_val_score(model, X, y, groups=g, scoring='accuracy', cv=cv, n_jobs=8, verbose=0)
    # return scores
    return mean(scores)

# define test conditions
ideal_cv = LeaveOneOut()
# define folds to test
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=1000)
# get the list of models to consider
models = get_models()
# collect results
ideal_results, cv_results = list(), list()
# evaluate each model
for model in models:
    # evaluate model using each test condition
    cv_mean = evaluate_model(cv, model)
    ideal_mean = evaluate_model(ideal_cv, model)
	# check for invalid results
    if isnan(cv_mean) or isnan(ideal_mean):
        continue
	# store results
    cv_results.append(cv_mean)
    ideal_results.append(ideal_mean)
	# summarize progress
    print('>%s: ideal=%.3f, cv=%.3f' % (type(model).__name__, ideal_mean, cv_mean))
    sys.stdout.flush()
    
# calculate the correlation between each test condition
corr, _ = pearsonr(cv_results, ideal_results)
print('Correlation: %.3f' % corr)
# scatter plot of results
pyplot.scatter(cv_results, ideal_results)
# plot the line of best fit
coeff, bias = polyfit(cv_results, ideal_results, 1)
line = coeff * asarray(cv_results) + bias
pyplot.plot(cv_results, line, color='r')
# label the plot
pyplot.title('5-fold CV vs LOOCV Mean Accuracy')
pyplot.xlabel('Mean Accuracy (10-fold CV)')
pyplot.ylabel('Mean Accuracy (LOOCV)')
# show the plot
pyplot.show()

NameError: name 'features_sample' is not defined

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

work = features_event.copy()
targets = pd.DataFrame(work[["gesture","subject","event"]], columns=["gesture","subject","event"])
targets.reset_index(inplace=True, drop=True)
work = work.drop(labels = ["gesture","subject", "event"], axis = 1)

#pca = PCA(n_components=3)
pca = PCA(.95)
principalComponents = pca.fit_transform(work)
#print(principalComponents[0])
#print(pca.explained_variance_ratio_)
#total = 0
#for tmp in pca.explained_variance_ratio_:
#    total += tmp
#print(total)
print(pca.n_features_)
print(pca.n_samples_)
#print(pca.noise_variance_)
print(pca.n_components_)
#print(pca.components_)
columns = []
for i in range(pca.n_components_):
    columns.append("principal component "+str(i+1))
#print(columns)
principalDf = pd.DataFrame(data = principalComponents, columns = columns)
finalDf = pd.concat([principalDf, targets], axis = 1)
#print(finalDf)

import matplotlib.cm as cm

fig = plt.figure(figsize = (70,70))
i = 0
for i in range(pca.n_components_):
    j = 0
    for j in range(pca.n_components_):
    
        ax = fig.add_subplot(pca.n_components_,pca.n_components_,pca.n_components_*i+j+1) 
        ax.set_xlabel('Principal Component ' + str(i + 1), fontsize = 15)
        ax.set_ylabel('Principal Component ' + str(j + 1), fontsize = 15)
        ax.set_title(str(i + 1) + 'x' + str(j + 1) + ' Component PCA', fontsize = 20)


        targets = finalDf['gesture'].unique()
        #colors = range(20)
        colors = cm.rainbow(np.linspace(0, 1, targets.size))
        #print(colors)
        for target, color in zip(targets,colors):
            indicesToKeep = finalDf['gesture'] == target
            colorarray = []
            for row in indicesToKeep.iteritems():
                if row[1] == True:
                    colorarray.append(color)
            ax.scatter( finalDf.loc[indicesToKeep, 'principal component '+ str(i + 1)]
                        , finalDf.loc[indicesToKeep, 'principal component '+ str(j + 1)]
                        , c = colorarray
                        #, cmap='Accent'
                        , s = 50)
            ax.legend(targets)
            ax.grid()
plt.show()

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit
from scipy.stats import pearsonr
from matplotlib import pyplot
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# create the dataset
def get_dataset():
    X = finalDf.copy().drop(labels = ["gesture","subject", "event"], axis = 1).to_numpy()
    y = finalDf.copy()["gesture"].to_list()
    return X, y
 
# retrieve the model to be evaluate
def get_model():
	model = LogisticRegression()
	return model

# get a list of models to evaluate
def get_models():
	models = list()
	models.append(LogisticRegression(penalty="l2",max_iter=100000,solver="sag"))
	models.append(RidgeClassifier(copy_X=True, max_iter=100000, solver="sag"))
	models.append(SGDClassifier(max_iter=100000))
	models.append(PassiveAggressiveClassifier(max_iter=100000))
	models.append(KNeighborsClassifier(n_neighbors=5))
	models.append(DecisionTreeClassifier())
	models.append(ExtraTreeClassifier())
	models.append(LinearSVC(max_iter=100000))
	models.append(SVC())
#	models.append(GaussianNB())
#	models.append(AdaBoostClassifier())
#	models.append(BaggingClassifier())
	models.append(RandomForestClassifier())
	models.append(ExtraTreesClassifier())
#	models.append(GaussianProcessClassifier(max_iter_predict=10000))
#	models.append(GradientBoostingClassifier())
#	models.append(LinearDiscriminantAnalysis())
#	models.append(QuadraticDiscriminantAnalysis())
	return models
 
# evaluate the model using a given test condition
def evaluate_model(cv, model):
	# get the dataset
	X, y = get_dataset()
	# evaluate the model
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=60, verbose=0)
	# return scores
	return mean(scores)

# define test conditions
ideal_cv = LeaveOneOut()
# define folds to test
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)
# get the list of models to consider
models = get_models()
# collect results
ideal_results, cv_results = list(), list()
# evaluate each model
for model in models:
	# evaluate model using each test condition
	cv_mean = evaluate_model(cv, model)
	ideal_mean = evaluate_model(ideal_cv, model)
	# check for invalid results
	if isnan(cv_mean) or isnan(ideal_mean):
		continue
	# store results
	cv_results.append(cv_mean)
	ideal_results.append(ideal_mean)
	# summarize progress
	print('>%s: ideal=%.3f, cv=%.3f' % (type(model).__name__, ideal_mean, cv_mean))
# calculate the correlation between each test condition
corr, _ = pearsonr(cv_results, ideal_results)
print('Correlation: %.3f' % corr)
# scatter plot of results
pyplot.scatter(cv_results, ideal_results)
# plot the line of best fit
coeff, bias = polyfit(cv_results, ideal_results, 1)
line = coeff * asarray(cv_results) + bias
pyplot.plot(cv_results, line, color='r')
# label the plot
pyplot.title('10-fold CV vs LOOCV Mean Accuracy')
pyplot.xlabel('Mean Accuracy (10-fold CV)')
pyplot.ylabel('Mean Accuracy (LOOCV)')
# show the plot
pyplot.show()

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy
 
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', init='glorot_uniform'):
	# create model
	model = Sequential()
	model.add(Dropout(0.4, input_shape=(pca.n_components_,)))
	model.add(Dense(12, input_dim=pca.n_components_, kernel_initializer=init, activation='relu'))
	model.add(Dense(12, kernel_initializer=init, activation='relu'))
	model.add(Dropout(0.4))
	model.add(Dense(targets.size, kernel_initializer=init, activation='relu'))
	model.add(Dense(targets.size, kernel_initializer=init, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
 
# fix random seed for reproducibility
seed = 1000
numpy.random.seed(seed)
# create the dataset
def get_dataset():
    X = finalDf.copy().drop(labels = ["gesture","subject", "event"], axis = 1).to_numpy()
    y = finalDf.copy()["gesture"].to_list()
    return X, y

X, Y = get_dataset()

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
cat_Y = np_utils.to_categorical(encoded_Y)
# create model
#model = KerasClassifier(build_fn=create_model, verbose=1)
# create model
model = KerasClassifier(build_fn=create_model, epochs=1500, batch_size=12, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())
